In [1]:
# read crd and store lipids in upper or lower membrane
from htmd import *
import numpy as np
from copy import deepcopy
import sys
import getopt

def read_crd(fname):
	from htmd.molecule.readers import Topology
	t = Topology()

	mol = Molecule()
	coords = list()
	nAtoms = 0
	current_resnum = 1
	up = []
	down = []

	with open(fname, 'r') as f:
		for line in f:
			pieces = line.split()
			if (not pieces[0].startswith("*") and len(pieces) > 7):
				if pieces[3] == 'P':
					if float(pieces[6]) > 0:
						up.append(pieces[1])
					else:
						down.append(pieces[1])
				if (int(pieces[1]) > current_resnum):
					mol._parseTopology(t,fname)
					mol.coords = deepcopy(np.array(coords, dtype=np.float32))
					current_resnum = int(pieces[1])
				t.resid.append(pieces[1])
				t.resname.append(pieces[2])
				t.name.append(pieces[3])
				coords.append([[float(pieces[4])],[float(pieces[5])],[float(pieces[6])]])
				t.segid.append(pieces[7])
				nAtoms = nAtoms + 1

	mol._parseTopology(t, fname)
	mol.coords = np.array(coords, dtype=np.float32)
	return(mol, up, down)


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.7.33 python[==3.5,==3.6]) is available. You are currently on (1.7.17). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.



In [2]:
# example molecules
m1 = read_crd('dlpe_mini/step5_assembly.crd')
m1[0].center
m2 = read_crd('sopa_1.crd') # has only 1 residue
m2[0].center

<bound method Molecule.center of <htmd.molecule.molecule.Molecule object at 0x7fc0ba79e518>
Molecule with 122 atoms and 1 frames
Atom field - altloc shape: (122,)
Atom field - atomtype shape: (122,)
Atom field - beta shape: (122,)
Atom field - chain shape: (122,)
Atom field - charge shape: (122,)
Atom field - coords shape: (122, 3, 1)
Atom field - element shape: (122,)
Atom field - insertion shape: (122,)
Atom field - masses shape: (122,)
Atom field - name shape: (122,)
Atom field - occupancy shape: (122,)
Atom field - record shape: (122,)
Atom field - resid shape: (122,)
Atom field - resname shape: (122,)
Atom field - segid shape: (122,)
Atom field - serial shape: (122,)
angles shape: (0, 3)
bonds shape: (0, 2)
box shape: (3, 1)
boxangles shape: (3, 1)
crystalinfo: None
dihedrals shape: (0, 4)
fileloc shape: (1, 2)
fstep: None
impropers shape: (0, 4)
reps: 
ssbonds shape: (0,)
step shape: (0,)
time shape: (0,)
topoloc: /home/cote/m/msi/project/sopa_1.crd
viewname: sopa_1>

In [4]:
m1[0].view(viewer='VMD')

In [ ]:
# store coord of residue to be rm of m1
coordp_m1 = m1[0].get('coords', 'resid 1 and name P')
# store coord of residue to be inserted of m1
coordp_m2 = m2[0].get('coords', 'resid 1 and name P')
# translate residue to be inserted to the final coord
m2[0].moveBy(coordp_m1-coordp_m2)
# remove residue
m1[0].remove('resid 1')
# insert residue
m1[0].append(m2[0])

In [ ]:
# random sampling (for future)
import random
population = m1[1] # population can be a list or a string
k = 3
rand = random.sample(population, k)